In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
# from tqdm import tqdm
from tqdm.notebook import tqdm
from glob import glob

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Conv2D, MaxPool2D, ZeroPadding2D, BatchNormalization, Input, DepthwiseConv2D, Add, LeakyReLU, ReLU
from tensorflow.keras.optimizers import Adam, SGD

# Audio
import librosa
import librosa.display as dsp
import IPython.display as ipd

# Augmentation
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

In [6]:
# Define Augmentation
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
])

In [3]:
train_path = glob('data/train/*.*')
len(train_path)

3805

In [4]:
# Audio augmentation
def path_to_mel(path, augmentation):
    audio, sr = librosa.load(path, sr=16000)

    if augmentation == True:
        audio = augment(samples=audio, sample_rate=16000)
        audio = np.hstack([audio, np.zeros(320000 - audio.shape[0])])

    elif augmentation == False:
        audio = np.hstack([audio, np.zeros(320000 - audio.shape[0])])
        
    mel = librosa.feature.melspectrogram(audio, sr=sr, n_mels=40)

    return np.array(mel, np.float64)

In [14]:
str('0'*(5 - len(train_path[1].split('/')[-1].split('.')[0])) + train_path[1].split('/')[-1].split('.')[0])

'00543'

In [6]:
data = []
label = []

for i in tqdm(train_path):
    if 
        data.append(path_to_mel(i))
        
        if i.split('/')[-2] == 'positive':
            label.append(1)
        else:
            label.append(0)

  0%|          | 0/3805 [00:00<?, ?it/s]

/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_48630/275320493.py:4: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = librosa.feature.melspectrogram(audio, sr=sr, n_mels=40)
/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_48630/275320493.py:4: FutureWarning: Pass y=[ 1.30398001e-11 -1.76966358e-11  2.29237931e-11 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = librosa.feature.melspectrogram(audio, sr=sr, n_mels=40)
/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_48630/275320493.py:4: FutureWarning: Pass y=[-3.23793006e-11  5.70472385e-11 -1.87248925e-10 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = librosa.feature.melspectrogram(aud

In [1]:
BATCH_SIZE = 16
SHUFFLE_BUFFER_SIZE = 100
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((data, label))
train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

NameError: name 'tf' is not defined

In [14]:
def residual_block(x, filters_in, filters_out):
    shortcut = x
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters_in, kernel_size=(1, 1), strides=(1, 1), padding="same",kernel_initializer='he_normal')(x)

    x = BatchNormalization()(x)
    x = ReLU()(x)    
    x = Conv2D(filters_in, kernel_size=(3, 3), strides=(1, 1), padding="same",kernel_initializer='he_normal')(x)

    x = BatchNormalization()(x)
    x = ReLU()(x)  
    x = Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1), padding="same",kernel_initializer='he_normal')(x)

    shortcut_channel = x.shape.as_list()[0]
    
    if shortcut_channel != filters_out:
        shortcut = Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1), padding="same",kernel_initializer='he_normal')(shortcut)
        
    x = Add()([x, shortcut])
    return ReLU()(x)

def build_model():

  inputs = tf.keras.layers.Input(shape=(40, 626, 1))

  outputs = Conv2D(16,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(inputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = residual_block(outputs, 16, 32)
  outputs = MaxPool2D((2,2))(outputs)
  outputs = residual_block(outputs, 32, 32)
  # outputs = residual_block(outputs, 32, 32)
  outputs = residual_block(outputs, 32, 64)
  outputs = MaxPool2D((2,2))(outputs)
  outputs = residual_block(outputs, 64, 64)
  # outputs = residual_block(outputs, 64, 64)
  # outputs = MaxPool2D((2,2))(outputs)

  outputs = GlobalAveragePooling2D()(outputs)
  # outputs = Flatten()(outputs)

  outputs = Dense(32,activation=None,kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = Dropout(0.5)(outputs)

  outputs = Dense(2,activation='softmax')(outputs)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

model = build_model()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 40, 626, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_34 (Conv2D)             (None, 40, 626, 16)  160         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_28 (BatchN  (None, 40, 626, 16)  64         ['conv2d_34[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
history = model.fit(train_ds, epochs=2)

Epoch 1/2


2022-07-03 07:19:12.808637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


238/238 [==============================] - 17s 66ms/step - loss: 0.1530 - accuracy: 0.9690
Epoch 2/2
134/238 [===============>..............] - ETA: 6s - loss: 0.0242 - accuracy: 1.0000

In [18]:
inf_path = glob('data/*.*')

inf_data = []
for i in tqdm(inf_path):
    inf_data.append(path_to_mel(i))

  0%|          | 0/5732 [00:00<?, ?it/s]

/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_39693/275320493.py:4: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = librosa.feature.melspectrogram(audio, sr=sr, n_mels=40)
/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_39693/275320493.py:4: FutureWarning: Pass y=[ 3.34500108e-08 -2.36135293e-07  4.83323220e-07 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = librosa.feature.melspectrogram(audio, sr=sr, n_mels=40)
/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_39693/275320493.py:4: FutureWarning: Pass y=[ 3.34639800e-08 -2.36143691e-07  4.83321060e-07 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = librosa.feature.melspectrogram(aud

In [11]:
inf_data_reshaped = []

for i in tqdm(inf_data):
    inf_data_reshaped.append(i.reshape(-1, 40, 626, 1))

  0%|          | 0/5732 [00:00<?, ?it/s]

In [20]:
# Model 추론
inf_result = []

for i in tqdm(inf_data_reshaped):
    preds = model.predict(i)
    score = tf.nn.softmax(preds[0])
    class_names = [0,1]

    inf_result.append(class_names[np.argmax(score)])

  0%|          | 0/5732 [00:00<?, ?it/s]

2022-07-03 01:22:47.390286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 12ms/step


In [21]:
np.max(inf_result)

1

In [22]:
submission = pd.read_csv('submit.csv')
submission = submission.drop('covid19', 1)
submission['covid19'] = inf_result
submission.to_csv('resnet_block_limited_data_mel.csv', index=False) 

/var/folders/8k/pkmq0wfd32q_r2nr5rcf2x1m0000gn/T/ipykernel_39693/2703957143.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  submission = submission.drop('covid19', 1)
